# Data Check-in

I guess I just wanted to take advantage of this clean slate by doing some *warehousing* (if you can call if that). In other words, I want to:

1. Identify all of the data which I still need/use 
2. Briefly check it over 
3. Clean (if necessary)
4. Organise and rename more pertinently

In [1]:
import pandas as pd

In [2]:
full_df = pd.read_csv('full_mocks.csv')
full_df

,Unnamed: 0,z,t/Gyr,M*,VIS,NISP_Y,NISP_J,NISP_H,LSST_u,LSST_g,...,2MASS_Ks,wircam_Ks,M_initial,Z,SFH,tau/Gyr,Av_law,Av,t_l,physical
0,0,0.00,0.001,0.9845,-13.0569,-16.6610,-18.1551,-19.3744,-2.3414,-7.1457,...,-20.0605,-20.0521,1.000000e+10,0.0,1.0,0.1,1.0,2.4,0.000000,True
1,1,0.01,0.001,0.9845,20.2890,16.6489,15.1359,13.9001,31.0738,26.2422,...,13.2033,13.2118,1.000000e+10,0.0,1.0,0.1,1.0,2.4,0.142799,True
2,2,0.02,0.001,0.9845,21.9111,18.2344,16.7026,15.4504,32.7630,27.9052,...,14.7433,14.7518,1.000000e+10,0.0,1.0,0.1,1.0,2.4,0.283535,True
3,3,0.03,0.001,0.9845,22.9071,19.1955,17.6448,16.3761,33.8259,28.9425,...,15.6582,15.6669,1.000000e+10,0.0,1.0,0.1,1.0,2.4,0.422245,True
4,4,0.04,0.001,0.9845,23.6479,19.9007,18.3312,17.0456,34.6281,29.7232,...,16.3177,16.3267,1.000000e+10,0.0,1.0,0.1,1.0,2.4,0.558968,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012817,1652150,0.01,13.000,0.5484,15.6757,14.2977,13.7890,13.3400,19.4627,17.5401,...,13.2999,13.3020,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.142799,True
1012818,1652151,0.02,13.000,0.5484,17.2263,15.8320,15.3213,14.8597,21.0540,19.1060,...,14.8092,14.8098,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.283535,True
1012819,1652152,0.03,13.000,0.5484,18.1522,16.7421,16.2287,15.7571,22.0174,20.0473,...,15.6949,15.6951,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.422245,True
1012820,1652153,0.04,13.000,0.5484,18.8222,17.3960,16.8793,16.3961,22.7193,20.7326,...,16.3243,16.3227,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.558968,True


So this appears to be the compilation of all of the cosmologically viable simulations! Great. Now let's get this into a notebook where I can start to make an attempt at some photometric errors a la Micol... First, lets drop the uncesessary filters and other bits:

In [3]:
todrop = ['Unnamed: 0', 'LSST_y', 'GAIA_Bp', 'GAIA_G',
       'GAIA_Rp', 'des_g', 'des_r', 'des_i', 'des_z', 'galex1500', 'galex2500',
       'MegaCam_u', 'MegaCam_r', 'kids_u', 'kids_g', 'kids_r', 'kids_i',
       'U_JKC', 'B_JKC', 'V_JKC', 'I_JKC', 'JPCam_g', 'PanSTA_i', 'Pan-STA_z',
       'HSC_z', '2MASS_J', '2MASS_H', '2MASS_Ks', 'wircam_Ks']

In [4]:
full_df.drop(columns=todrop, inplace=True)

Now we need to get rid of all objects where **any** magnitude is greater than 30, since this is unrealistically faint for observation (see limiting magnitudes). This has proved more problematic than expected! I'm uncertain as to whether I should be doing this prior to applying obs errors to the readings. In my mind, though (which isn't up to much), it maked sense to eliminate these objects very quickly, right? 

30 is way fainter than any of the limiting magnitudes so I assume that this is going to even exceed the observed frame magnitudes.

And on another point, leaving in m > 30 as say -99. just doesn't do us any favours, since we want to compare the effectiveness of filter sets. For example, if we have a bunch of objects with u>30 in our full-filter fit, we might as well be doing them as a no-u fit...

So here, let's take only objects where **all** the magnitudes are under 30 and save this as a 'fully observable' set of galaxies.

In [6]:
observable_df = full_df[(full_df['NISP_Y'] < 30) & (full_df['NISP_J'] < 30) & (full_df['NISP_H'] < 30) & (full_df['LSST_u'] < 30) & (full_df['LSST_g'] < 30) & (full_df['LSST_r'] < 30) & (full_df['LSST_i'] < 30) & (full_df['LSST_z'] < 30) & (full_df['VIS'] < 30)]

In [12]:
observable_df = observable_df[(observable_df['NISP_Y'] > 3) & (observable_df['NISP_J'] > 3) & (observable_df['NISP_H'] > 3) & (observable_df['LSST_u'] > 3) & (observable_df['LSST_g'] > 3) & (observable_df['LSST_r'] > 3) & (observable_df['LSST_i'] > 3) & (observable_df['LSST_z'] > 3) & (observable_df['VIS'] > 3)]

I also decided to filter out all galaxies with any magnitude less that 3 because... well that would be a bit mad.

In [20]:
observable_df

,z,t/Gyr,M*,VIS,NISP_Y,NISP_J,NISP_H,LSST_u,LSST_g,LSST_r,LSST_i,LSST_z,M_initial,Z,SFH,tau/Gyr,Av_law,Av,t_l,physical
11778,0.01,0.001,0.9845,17.3743,14.8193,13.8647,13.1298,24.4559,21.2513,18.8165,17.1871,16.1598,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.142799,True
11779,0.02,0.001,0.9845,18.9609,16.3787,15.4099,14.6630,26.0896,22.8653,20.4177,18.7772,17.7395,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.283535,True
11780,0.03,0.001,0.9845,19.9219,17.3139,16.3305,15.5716,27.0969,23.8540,21.3933,19.7427,18.6937,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.422245,True
11781,0.04,0.001,0.9845,20.6274,17.9932,16.9954,16.2240,27.8444,24.5863,22.1115,20.4527,19.3919,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.558968,True
11782,0.05,0.001,0.9845,21.1922,18.5324,17.5203,16.7361,28.4497,25.1780,22.6875,21.0227,19.9497,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.693734,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012817,0.01,13.000,0.5484,15.6757,14.2977,13.7890,13.3400,19.4627,17.5401,16.2592,15.4677,14.9378,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.142799,True
1012818,0.02,13.000,0.5484,17.2263,15.8320,15.3213,14.8597,21.0540,19.1060,17.8141,17.0149,16.4871,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.283535,True
1012819,0.03,13.000,0.5484,18.1522,16.7421,16.2287,15.7571,22.0174,20.0473,18.7439,17.9375,17.4130,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.422245,True
1012820,0.04,13.000,0.5484,18.8222,17.3960,16.8793,16.3961,22.7193,20.7326,19.4171,18.6058,18.0817,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.558968,True


300k objects, whew. So I think I'm going to test out applying photometric errors to a subset of these first (just to test it out) and then hopefully come back here and apply the errors to all of these (all things going well). So let's reorder, save, subset, save and take it to the other notebook.

In [21]:
observable_df = observable_df[['LSST_u', 'LSST_g', 'LSST_r', 'LSST_i', 'LSST_z', 'VIS', 'NISP_Y', 'NISP_J', 'NISP_H', 'z', 't/Gyr', 'M*', 'M_initial', 'Z', 'SFH', 'tau/Gyr', 'Av_law', 'Av', 't_l', 'physical']]

In [22]:
observable_df

,LSST_u,LSST_g,LSST_r,LSST_i,LSST_z,VIS,NISP_Y,NISP_J,NISP_H,z,t/Gyr,M*,M_initial,Z,SFH,tau/Gyr,Av_law,Av,t_l,physical
11778,24.4559,21.2513,18.8165,17.1871,16.1598,17.3743,14.8193,13.8647,13.1298,0.01,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.142799,True
11779,26.0896,22.8653,20.4177,18.7772,17.7395,18.9609,16.3787,15.4099,14.6630,0.02,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.283535,True
11780,27.0969,23.8540,21.3933,19.7427,18.6937,19.9219,17.3139,16.3305,15.5716,0.03,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.422245,True
11781,27.8444,24.5863,22.1115,20.4527,19.3919,20.6274,17.9932,16.9954,16.2240,0.04,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.558968,True
11782,28.4497,25.1780,22.6875,21.0227,19.9497,21.1922,18.5324,17.5203,16.7361,0.05,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.693734,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012817,19.4627,17.5401,16.2592,15.4677,14.9378,15.6757,14.2977,13.7890,13.3400,0.01,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.142799,True
1012818,21.0540,19.1060,17.8141,17.0149,16.4871,17.2263,15.8320,15.3213,14.8597,0.02,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.283535,True
1012819,22.0174,20.0473,18.7439,17.9375,17.4130,18.1522,16.7421,16.2287,15.7571,0.03,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.422245,True
1012820,22.7193,20.7326,19.4171,18.6058,18.0817,18.8222,17.3960,16.8793,16.3961,0.04,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.558968,True


In [21]:
observable_df.to_csv('observable_full.csv', index=False)

In [24]:
sample_df = observable_df.sample(100, random_state=69).copy()

In [25]:
sample_df.to_csv('euc_phys_n100.csv')

## Making Observable with photometric errors (fully)

In [2]:
observable_df = pd.read_csv('observable_full.csv')
observable_df

,For_key_ID,LSST_u,LSST_g,LSST_r,LSST_i,LSST_z,VIS,NISP_Y,NISP_J,NISP_H,...,t/Gyr,M*,M_initial,Z,SFH,tau/Gyr,Av_law,Av,t_l,physical
0,11778,24.4559,21.2513,18.8165,17.1871,16.1598,17.3743,14.8193,13.8647,13.1298,...,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.142799,True
1,11779,26.0896,22.8653,20.4177,18.7772,17.7395,18.9609,16.3787,15.4099,14.6630,...,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.283535,True
2,11780,27.0969,23.8540,21.3933,19.7427,18.6937,19.9219,17.3139,16.3305,15.5716,...,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.422245,True
3,11781,27.8444,24.5863,22.1115,20.4527,19.3919,20.6274,17.9932,16.9954,16.2240,...,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.558968,True
4,11782,28.4497,25.1780,22.6875,21.0227,19.9497,21.1922,18.5324,17.5203,16.7361,...,0.001,0.9845,1.000000e+10,0.0,1.0,0.1,1.0,2.0,0.693734,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299775,1012817,19.4627,17.5401,16.2592,15.4677,14.9378,15.6757,14.2977,13.7890,13.3400,...,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.142799,True
299776,1012818,21.0540,19.1060,17.8141,17.0149,16.4871,17.2263,15.8320,15.3213,14.8597,...,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.283535,True
299777,1012819,22.0174,20.0473,18.7439,17.9375,17.4130,18.1522,16.7421,16.2287,15.7571,...,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.422245,True
299778,1012820,22.7193,20.7326,19.4171,18.6058,18.0817,18.8222,17.3960,16.8793,16.3961,...,13.000,0.5484,1.000000e+10,-0.3,1.0,10.0,1.0,1.0,0.558968,True


### Processing all of these galaxies to have photometric errors applied (as a copy)

In [3]:
limiting_mags = {
    'LSST_u': 24.2,
    'LSST_g': 24.5,
    'LSST_r': 23.9,
    'LSST_i': 23.6,
    'LSST_z': 23.4,
    'VIS': 24.5,
    'NISP_Y': 23.0,
    'NISP_J': 23.0,
    'NISP_H': 23.0
}

In [4]:
limiting_fluxes = limiting_mags.copy()

In [5]:
for i in limiting_fluxes.keys():
    limiting_fluxes[i] = 10**(-0.4*(limiting_mags[i] - 23.9))

In [6]:
limiting_fluxes

{'LSST_u': 0.7585775750291832,
 'LSST_g': 0.5754399373371561,
 'LSST_r': 1.0,
 'LSST_i': 1.3182567385564037,
 'LSST_z': 1.5848931924611136,
 'VIS': 0.5754399373371561,
 'NISP_Y': 2.29086765276777,
 'NISP_J': 2.29086765276777,
 'NISP_H': 2.29086765276777}

In [7]:
lim_flux_err = limiting_fluxes.copy()

In [8]:
SN = 10

In [9]:
for i in lim_flux_err.keys():
    lim_flux_err[i] = lim_flux_err[i]/SN

In [10]:
lim_flux_err

{'LSST_u': 0.07585775750291832,
 'LSST_g': 0.05754399373371562,
 'LSST_r': 0.1,
 'LSST_i': 0.13182567385564037,
 'LSST_z': 0.15848931924611137,
 'VIS': 0.05754399373371562,
 'NISP_Y': 0.22908676527677702,
 'NISP_J': 0.22908676527677702,
 'NISP_H': 0.22908676527677702}

In [13]:
import numpy as np

In [14]:
lim_mag_err = lim_flux_err.copy()
for i in lim_flux_err.keys():
    lim_mag_err[i] = (2.5/np.log(10.))*np.abs(lim_flux_err[i]/limiting_fluxes[i])

In [15]:
lim_mag_err

{'LSST_u': 0.10857362047581293,
 'LSST_g': 0.10857362047581294,
 'LSST_r': 0.10857362047581294,
 'LSST_i': 0.10857362047581294,
 'LSST_z': 0.10857362047581294,
 'VIS': 0.10857362047581294,
 'NISP_Y': 0.10857362047581294,
 'NISP_J': 0.10857362047581294,
 'NISP_H': 0.10857362047581294}

In [16]:
phot_errors_df = observable_df.copy()

In [17]:
for name in lim_flux_err.keys():
    mag = phot_errors_df[name]
    flux = 10**(-0.4*(mag - 23.9))
    noise = np.random.normal(0, lim_flux_err[name])
    obs_flux = flux + noise
    flux_err = lim_flux_err[name]
    obs_mag = -2.5*np.log(obs_flux) + 23.9
    dmag=(2.5/np.log(10.))*np.abs(flux_err/flux)
    phot_errors_df[name] = obs_mag
    phot_errors_df['delta_'+ name] = dmag

/home/chris/Documents/euclid/euclid_env/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [19]:
phot_errors_df.dropna(inplace=True)

In [20]:
phot_errors_df

,For_key_ID,LSST_u,LSST_g,LSST_r,LSST_i,LSST_z,VIS,NISP_Y,NISP_J,NISP_H,...,physical,delta_LSST_u,delta_LSST_g,delta_LSST_r,delta_LSST_i,delta_LSST_z,delta_VIS,delta_NISP_Y,delta_NISP_J,delta_NISP_H
0,11778,25.605317,17.799151,12.194722,8.441872,6.076895,8.874203,2.990675,0.792832,-0.899277,...,True,0.137431,0.005448,0.001005,0.000296,0.000138,0.000153,0.000058,0.000024,0.000012
1,11779,31.988718,21.508721,15.881329,12.099544,9.712201,12.528087,6.580555,4.350673,2.631122,...,True,0.618820,0.024090,0.004393,0.001278,0.000591,0.000661,0.000244,0.000100,0.000050
8,12379,24.925312,16.971933,11.198652,7.304799,4.808424,7.692217,1.485123,-0.939385,-2.840600,...,True,0.108754,0.003912,0.000675,0.000187,0.000083,0.000096,0.000032,0.000012,0.000006
9,12380,30.439491,20.689618,14.893183,10.971860,8.455835,11.354856,5.084561,2.627493,0.695054,...,True,0.493588,0.017343,0.002959,0.000814,0.000357,0.000413,0.000134,0.000050,0.000023
17,12980,25.208541,16.750316,10.666306,6.510137,3.747985,6.794865,-0.074356,-3.029892,-5.485587,...,True,0.120017,0.003580,0.000546,0.000136,0.000054,0.000067,0.000017,0.000005,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299775,1012817,13.686678,9.255724,6.306400,4.483685,3.263565,4.962888,1.789738,0.618529,-0.415269,...,True,0.001383,0.000179,0.000095,0.000061,0.000045,0.000032,0.000036,0.000022,0.000015
299776,1012818,17.363943,12.861131,9.886663,8.045529,6.830307,8.533397,5.322132,4.146676,3.084061,...,True,0.005989,0.000755,0.000399,0.000252,0.000186,0.000134,0.000147,0.000092,0.000060
299777,1012819,19.606889,15.028173,12.027560,10.168633,8.961114,10.665574,7.416913,6.235862,5.150556,...,True,0.014544,0.001797,0.000940,0.000590,0.000437,0.000314,0.000341,0.000212,0.000138
299778,1012820,21.261617,16.605557,13.577590,11.705577,10.499137,12.208625,8.921406,7.733664,6.622129,...,True,0.027762,0.003379,0.001748,0.001092,0.000810,0.000582,0.000622,0.000387,0.000248


In [21]:
phot_errors_df.to_csv('all_observed.csv', index=False)

In [22]:
larger_sample_df = phot_errors_df.sample(1000, random_state=69).copy()

In [23]:
larger_sample_df.to_csv('euc_obs_n1000.csv')

### Now to make catalogues (4) for each of the filter setups.

In [25]:
larger_sample_df.columns

Index(['For_key_ID', 'LSST_u', 'LSST_g', 'LSST_r', 'LSST_i', 'LSST_z', 'VIS',
       'NISP_Y', 'NISP_J', 'NISP_H', 'z', 't/Gyr', 'M*', 'M_initial', 'Z',
       'SFH', 'tau/Gyr', 'Av_law', 'Av', 't_l', 'physical', 'delta_LSST_u',
       'delta_LSST_g', 'delta_LSST_r', 'delta_LSST_i', 'delta_LSST_z',
       'delta_VIS', 'delta_NISP_Y', 'delta_NISP_J', 'delta_NISP_H'],
      dtype='object')

In [26]:
full_filts = larger_sample_df.drop(columns=['For_key_ID', 'z', 't/Gyr', 'M*', 'M_initial', 'Z',
       'SFH', 'tau/Gyr', 'Av_law', 'Av', 't_l', 'physical'])

In [28]:
full_filts['z'] = larger_sample_df['z']

In [29]:
full_filts.insert(loc=0, column='ID', value=np.arange(1, len(full_filts)+1))

In [30]:
full_filts

,ID,LSST_u,LSST_g,LSST_r,LSST_i,LSST_z,VIS,NISP_Y,NISP_J,NISP_H,delta_LSST_u,delta_LSST_g,delta_LSST_r,delta_LSST_i,delta_LSST_z,delta_VIS,delta_NISP_Y,delta_NISP_J,delta_NISP_H,z
166926,1,28.719878,25.401189,21.493210,20.948536,21.007141,21.374009,20.931209,21.322368,22.030668,0.344322,0.115826,0.041519,0.047046,0.060096,0.022503,0.086777,0.093662,0.107598,4.77
130815,2,7.905253,8.517532,9.088136,9.229868,9.600184,9.203728,10.118776,10.487087,10.619535,0.000137,0.000133,0.000290,0.000405,0.000565,0.000175,0.001006,0.001164,0.001225,0.69
172088,3,28.420486,26.783728,26.661817,25.428297,25.036981,26.944591,24.533737,24.392211,24.431951,0.319598,0.203913,0.331470,0.435516,0.542606,0.191904,0.684360,0.369662,0.246811,2.39
153404,4,26.182985,23.746710,19.892765,19.213896,19.114421,19.626215,19.035612,19.246821,19.734782,0.166389,0.059271,0.021874,0.022707,0.026620,0.011245,0.037765,0.039584,0.045299,4.86
229877,5,27.089987,22.066277,19.901048,19.628434,19.269790,19.788060,19.120893,19.325972,19.631161,0.220864,0.030136,0.021946,0.026968,0.028416,0.011993,0.039160,0.040889,0.043524,4.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48245,996,13.432322,13.604540,13.649427,13.978850,14.250585,13.851010,14.490162,14.682879,15.218563,0.001249,0.001017,0.001799,0.002716,0.003651,0.001122,0.005824,0.006254,0.007672,1.85
291034,997,20.452421,17.281212,15.056649,13.868763,12.866052,14.205811,11.477282,10.703143,9.992687,0.020262,0.004428,0.003159,0.002599,0.002092,0.001292,0.001734,0.001269,0.000954,0.11
156544,998,9.700758,9.220035,8.636603,8.501022,8.379209,8.554812,8.042584,7.766353,7.628374,0.000281,0.000176,0.000242,0.000303,0.000347,0.000135,0.000438,0.000392,0.000371,0.26
120652,999,20.317921,19.791681,19.277236,17.797862,17.301947,18.380333,16.900157,16.531292,15.668990,0.019224,0.012101,0.017097,0.012701,0.012574,0.006846,0.015515,0.013154,0.009176,0.85


In [41]:
np.savetxt(r'all_1000.cat', full_filts.values, fmt=' '.join(['%i'] + ['%1.4f']*19))

In [32]:
no_u_cat = full_filts.drop(columns=['LSST_u', 'delta_LSST_u'])

In [34]:
np.savetxt(r'no_u_1000.cat', no_u_cat.values, fmt=' '.join(['%i'] + ['%1.4f']*17))

In [35]:
gnd_cat = full_filts.drop(columns=['VIS',
       'NISP_Y', 'NISP_J', 'NISP_H','delta_VIS', 'delta_NISP_Y', 'delta_NISP_J', 'delta_NISP_H'])

In [37]:
np.savetxt(r'gnd_1000.cat', gnd_cat.values, fmt=' '.join(['%i'] + ['%1.4f']*11))

In [38]:
euc_cat = full_filts.drop(columns=['LSST_u', 'LSST_g', 'LSST_r', 'LSST_i', 'LSST_z', 'delta_LSST_u',
       'delta_LSST_g', 'delta_LSST_r', 'delta_LSST_i', 'delta_LSST_z'])

In [40]:
np.savetxt(r'euc_1000.cat', euc_cat.values, fmt=' '.join(['%i'] + ['%1.4f']*9))